In [180]:
from typing import Optional

import clingo
import clingo.ast

In [181]:
program = """

domA(0).

domA(A) :- domA(A).

"""

In [182]:
unground_reified = """

rule(head(literal(sign(0), atom(symbol(function(name("domA"), arguments(term(symbol(0))), external(0)))))), body(elements())).
rule(head(literal(sign(0), atom(symbol(function(name("domA"), arguments(variable(name("A"))), external(0)))))), body(elements(literal(sign(0), atom(symbol(function(name("domA"), arguments(variable(name("A"))), external(0)))))))).


"""

In [183]:
ctl = clingo.Control()
ctl.configuration.solve.models = 0


In [184]:
ctl.add('base', [], unground_reified)

In [185]:
ctl.ground([('base', [])])

In [186]:
with ctl.solve(yield_=True) as solve_handle:
    models = []
    for model in solve_handle:
        symbols = model.symbols(atoms=True)
        models.append(symbols)

In [187]:
print(' '.join(map(str, models[0])))

rule(head(literal(sign(0),atom(symbol(function(name("domA"),arguments(term(symbol(0))),external(0)))))),body(elements)) rule(head(literal(sign(0),atom(symbol(function(name("domA"),arguments(variable(name("A"))),external(0)))))),body(elements(literal(sign(0),atom(symbol(function(name("domA"),arguments(variable(name("A"))),external(0))))))))


In [188]:
pos = clingo.ast.Position('<string>', 1, 1)
loc = clingo.ast.Location(pos, pos)
_false = clingo.ast.BooleanConstant(False)

In [190]:
def translate_variable(variable: clingo.Symbol):
    print(variable)
    assert variable.match('variable', 1)
    name = variable.arguments[0]
    name_node = translate_name(name)
    variable_node = clingo.ast.Variable(loc, name_node)
    return variable_node


def translate_term(term: clingo.Symbol):
    print(term)
    assert term.match('term', 1)
    symbol = term.arguments[0]
    term_node = translate_symbol(symbol)
    return term_node


def translate_arguments(arguments: clingo.Symbol):
    print(arguments)
    assert arguments.type is clingo.SymbolType.Function and arguments.name == 'arguments'
    arguments_node = tuple(translate(argument) for argument in arguments.arguments)
    return arguments_node


def translate_elements(elements: clingo.Symbol):
    print(elements)
    assert elements.type is clingo.SymbolType.Function and elements.name == 'elements'
    elements_node = tuple(translate(argument) for argument in elements.arguments)
    return elements_node


def translate_name(name: clingo.Symbol):
    print(name)
    assert name.match('name', 1)
    assert name.arguments[0].type is clingo.SymbolType.String
    name_node = name.arguments[0].string
    return name_node


def translate_external(external: clingo.Symbol):
    print(external)
    assert external.match('external', 1)
    assert external.arguments[0].type is clingo.SymbolType.Number
    external_node = external.arguments[0].number
    return external_node


def translate_function(function: clingo.Symbol):
    print(function)
    assert function.match('function', 3)
    name = function.arguments[0]
    arguments = function.arguments[1]
    external = function.arguments[2]
    name_node = translate_name(name)
    arguments_node = translate_arguments(arguments)
    external_node = translate_external(external)
    function_node = clingo.ast.Function(loc, name_node, arguments_node, external_node)
    return function_node


def translate_symbol(symbol: clingo.Symbol):
    print(symbol)
    assert symbol.match('symbol', 1)
    child = symbol.arguments[0]
    child_node = translate(child)
    return child_node


def translate_atom(atom: clingo.Symbol):
    print(atom)
    assert atom.match('atom', 1)
    symbol = atom.arguments[0]
    symbol_node = translate_symbol(symbol)
    atom_node = clingo.ast.SymbolicAtom(symbol_node)
    return atom_node


def translate_sign(sign: clingo.Symbol):
    print(sign)
    assert sign.match('sign', 1)
    sign_node = sign.arguments[0].number
    return sign_node


def translate_literal(literal: clingo.Symbol):
    print(literal)
    assert literal.match('literal', 2)
    sign = literal.arguments[0]
    atom = literal.arguments[1]
    sign_node = translate_sign(sign)
    atom_node = translate_atom(atom)
    literal_node = clingo.ast.Literal(loc, sign_node, atom_node)
    return literal_node


def translate_head(head: Optional[clingo.Symbol] = None):
    print(head)
    assert head.match('head', 1)
    child = head.arguments[0]
    head_node = translate(child)
    return head_node


def translate_body(body: Optional[clingo.Symbol] = None):
    print(body)
    assert body.match('body', 1)
    elements = body.arguments[0]
    elements_node = translate_elements(elements)
    return elements_node


def translate_rule(rule: clingo.Symbol):
    print(rule)
    assert rule.match('rule', 2)
    head = rule.arguments[0]
    body = rule.arguments[1]
    head_node = translate_head(head)
    body_node = translate_body(body)
    rule_node = clingo.ast.Rule(loc, head_node, body_node)
    return rule_node


def translate(symbol: clingo.Symbol):
    print(symbol)
    if symbol.type is clingo.SymbolType.Function:
        if symbol.name == 'literal':
            return translate_literal(symbol)
        elif symbol.name == 'function':
            return translate_function(symbol)
        elif symbol.name == 'term':
            return translate_term(symbol)
        elif symbol.name == 'variable':
            return translate_variable(symbol)
    else:
        return clingo.ast.SymbolicTerm(loc, symbol)




In [191]:
prg = []

for symbol in models[0]:
    prg.append(translate_rule(symbol))

rule(head(literal(sign(0),atom(symbol(function(name("domA"),arguments(term(symbol(0))),external(0)))))),body(elements))
head(literal(sign(0),atom(symbol(function(name("domA"),arguments(term(symbol(0))),external(0))))))
literal(sign(0),atom(symbol(function(name("domA"),arguments(term(symbol(0))),external(0)))))
literal(sign(0),atom(symbol(function(name("domA"),arguments(term(symbol(0))),external(0)))))
sign(0)
atom(symbol(function(name("domA"),arguments(term(symbol(0))),external(0))))
symbol(function(name("domA"),arguments(term(symbol(0))),external(0)))
function(name("domA"),arguments(term(symbol(0))),external(0))
function(name("domA"),arguments(term(symbol(0))),external(0))
name("domA")
arguments(term(symbol(0)))
term(symbol(0))
term(symbol(0))
symbol(0)
0
external(0)
body(elements)
elements
rule(head(literal(sign(0),atom(symbol(function(name("domA"),arguments(variable(name("A"))),external(0)))))),body(elements(literal(sign(0),atom(symbol(function(name("domA"),arguments(variable(name("

In [192]:
print('\n'.join(map(str, prg)))

domA(0).
domA(A) :- domA(A).
